In [3]:
# My code: 
url = 'https://bnf.libguides.com/signets_prisme/sectoriels'

import pandas as pd
from datetime import datetime
import locale

import requests
from bs4 import BeautifulSoup



In [4]:
# Test response
response = requests.get(url)
response

<Response [200]>

In [4]:
soup = BeautifulSoup(response.content, "html.parser")

In [5]:
def extract_bnf_info(soup):
    # your code goes here
    
    # Lists to store extracted data
    bnf_codes = []
    bnf_names = []
    web_sites_description = []
    web_sites = []

    # Find the table containing the data and extract the rows
    table = soup.find("table")
    rows = table.find_all("tr")

    # Iteration per row to extract the data
    for row in rows:
        cells = row.find_all("td")
        # Extract the 3 columns at once:
        if len(cells) >= 3: #checking that there are at least my 3 columns
            bnf_codes.append(cells[0].text.strip())
            bnf_names.append(cells[1].text.strip())
            
            # Extracting the text and the link for the 3rd column
            text_with_links = cells[2].find_all("a")
            if text_with_links: # Condition to avoid the error with the empty links
                text_parts = [] # creating 2 lists to get the pieces if more than one link and description
                links = []
                for tag in text_with_links: #going through each tag to add the description and link
                    text_parts.append(tag.text.strip())
                    links.append(tag.get("href"))
                # concatenating the pieces of the lists together so it appears in the same cells
                text = "* ".join(text_parts)
                link = "* ".join(links)
            else:
                text = None
                link = None

            web_sites_description.append(text)
            web_sites.append(link)
             
    # Create DataFrame
    df = pd.DataFrame({
        'bnf_codes': bnf_codes,
        'bnf_names': bnf_names,
        'web_sites_description': web_sites_description,
        'web_sites': web_sites
        })
   
    return df

In [6]:
#extracting the data from the web site
df= extract_bnf_info(soup)
df

,bnf_codes,bnf_names,web_sites_description,web_sites
0,084.11,"Agriculture, sylviculture,\n\t\t\tpêche, élevage",Abiosol* AFPF* Agence Bio* Agreste* Associatio...,https://devenirpaysan-idf.org/* https://afpf-a...
1,084.111,"Espaces verts, jardineries\n\t\t\tet animaleries",FFP - Fédération française du paysage* UNEP - ...,https://www.f-f-p.org/annuaire/* https://www.l...
2,084.12,Industrie et distribution\n\t\t\talimentaires,Annuaire Rungis* CEDUS - Cultures Sucre* CTIFL...,https://www.rungisinternational.com/annuaire/*...
3,084.121,"Boissons, vins et spiritueux",Fédération française des spiritueux* Vinexposi...,http://www.spiritueux.fr* https://vinexposium-...
4,084.13,Industries extractives\n\t\t\t(hors énergie),"A3M - Alliance des minerais, minéraux et métau...",https://www.a3ms.fr/* https://unctad.org/fr/no...
5,084.17,"Industries textiles,\n\t\t\thabillement, cuir",Alliance France Cuir* FashionNetwork* Fédérati...,https://alliancefrancecuir.org/* https://fr.fa...
6,084.18,"Bijouterie, horlogerie,\n\t\t\torfèvrerie",Francéclat* * FFHBI - Fédération française hor...,https://www.franceclat.fr/category/actualites/...
7,084.20,Industries du bois\n\t\t\t(hors meubles),Bois international* France bois forêt - Interp...,https://www.leboisinternational.com/* http://o...
8,084.22,"Papier, carton, imprimerie",CAP* Copacel - Union Française des Industries ...,https://www.cap-fede.fr/* http://www.copacel.f...
9,084.23,Emballage,CNE - Conseil National de l'Emballage* ELIPSO ...,https://conseil-emballage.org/* https://www.el...


In [7]:
# Clean up of the bnf_names column
" ".join(df['bnf_names'][0].split())
df['bnf_names'] = df['bnf_names'].apply(lambda x : " ".join(x.split()))

In [8]:
# Clean up and conversion into list of the web_sites_description and web_sites columns
df['web_sites_description'] = df['web_sites_description'].apply(lambda x : str(x).split('* '))
df['web_sites'] = df['web_sites'].apply(lambda x : str(x).split('* '))
df

,bnf_codes,bnf_names,web_sites_description,web_sites
0,084.11,"Agriculture, sylviculture, pêche, élevage","[Abiosol, AFPF, Agence Bio, Agreste, Associati...","[https://devenirpaysan-idf.org/, https://afpf-..."
1,084.111,"Espaces verts, jardineries et animaleries","[FFP - Fédération française du paysage, UNEP -...","[https://www.f-f-p.org/annuaire/, https://www...."
2,084.12,Industrie et distribution alimentaires,"[Annuaire Rungis, CEDUS - Cultures Sucre, CTIF...",[https://www.rungisinternational.com/annuaire/...
3,084.121,"Boissons, vins et spiritueux","[Fédération française des spiritueux, Vinexpos...","[http://www.spiritueux.fr, https://vinexposium..."
4,084.13,Industries extractives (hors énergie),"[A3M - Alliance des minerais, minéraux et méta...","[https://www.a3ms.fr/, https://unctad.org/fr/n..."
5,084.17,"Industries textiles, habillement, cuir","[Alliance France Cuir, FashionNetwork, Fédérat...","[https://alliancefrancecuir.org/, https://fr.f..."
6,084.18,"Bijouterie, horlogerie, orfèvrerie","[Francéclat, , FFHBI - Fédération française ho...",[https://www.franceclat.fr/category/actualites...
7,084.20,Industries du bois (hors meubles),"[Bois international, France bois forêt - Inter...","[https://www.leboisinternational.com/, http://..."
8,084.22,"Papier, carton, imprimerie","[CAP, Copacel - Union Française des Industries...","[https://www.cap-fede.fr/, http://www.copacel...."
9,084.23,Emballage,"[CNE - Conseil National de l'Emballage, ELIPSO...","[https://conseil-emballage.org/, https://www.e..."


In [56]:
# Creating an 'exploded' dataframe in case it is the one needed
df_exploded = df.explode('web_sites_description').explode('web_sites')


In [57]:
df_exploded.reset_index(drop = True, inplace = True)

In [58]:
df_exploded

,bnf_codes,bnf_names,web_sites_description,web_sites
0,084.11,"Agriculture, sylviculture, pêche, élevage",Abiosol,https://devenirpaysan-idf.org/
1,084.11,"Agriculture, sylviculture, pêche, élevage",Abiosol,https://afpf-asso.fr/
2,084.11,"Agriculture, sylviculture, pêche, élevage",Abiosol,http://www.agencebio.org/
3,084.11,"Agriculture, sylviculture, pêche, élevage",Abiosol,https://agreste.agriculture.gouv.fr/agreste-web/
4,084.11,"Agriculture, sylviculture, pêche, élevage",Abiosol,https://www.agroforesterie.fr/
...,...,...,...,...
1957,084.636,Autres activités liées à l'art,Syndicat national des antiquaires,https://www.conseildesventes.fr/
1958,084.636,Autres activités liées à l'art,Syndicat national des antiquaires,https://www2.deloitte.com/lu/en/pages/art-fina...
1959,084.636,Autres activités liées à l'art,Syndicat national des antiquaires,http://www.guideartservices.fr/
1960,084.636,Autres activités liées à l'art,Syndicat national des antiquaires,https://www.sitem.fr/


In [59]:
# exporting the two dataframes to csv 

df.to_csv("bnf_links.csv")
df_exploded.to_csv("bnf_links_exploded.csv")


In [60]:
#export to my SQL
# Name of database: 'match_project'

from sqlalchemy import create_engine 
import pymysql.cursors
import os
# import getpass
import urllib.parse


urllib.parse.quote_plus('!') # parse to 'remove-change' the weird characters in our password, here it is '!'
urllib.parse.quote_plus('é')
pw_raw = 'Alsimar10' # os.getenv('mysql_andy') # I had to put my.sql password
pw = urllib.parse.quote_plus(pw_raw)

#engine = sqlalchemy.create_engine(f"mysql://{username}:{password}@{host}:{port}")

connection_string = 'mysql+pymysql://root:' + pw + '@localhost:3306/' # we connect to our computer, if we connect to a company
                     # this is the driver                             # data base, then we need an IP address
engine = create_engine(connection_string)

with engine.connect() as conn:                                   # we are creating an alias
    conn.execute(f"CREATE DATABASE IF NOT EXISTS match_project") # we are creating a new name of database "match_project"

In [61]:
df_exploded.to_sql('bnf_exploded',engine, 'match_project', if_exists='replace', index=False) # 'pushing' the data


1962

In [5]:
# To combine with other table from insee
# Codes APE from https://www.insee.fr/fr/information/2028155
naf2_agreg = pd.read_excel("niv_agreg_naf_rev_2_jav.xls")
naf2 = pd.read_excel("table_NAF2-NA_jav.xls")
naf2

,SOUS- CLASSE,INTITULE DE LA NAF rév. 2,CLASSE,GROUPE,A 129,DIVISION,A 64,A 38,A 17,A 10,...,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,01.11Z,"Culture de céréales (à l'exception du riz), de...",1.11,1.1,A01Z,1,AZ1,AZ,AZ,AZ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01.11Z
1,01.12Z,Culture du riz,1.12,1.1,A01Z,1,AZ1,AZ,AZ,AZ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01.12Z
2,01.13Z,"Culture de légumes, de melons, de racines et d...",1.13,1.1,A01Z,1,AZ1,AZ,AZ,AZ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01.13Z
3,01.14Z,Culture de la canne à sucre,1.14,1.1,A01Z,1,AZ1,AZ,AZ,AZ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01.14Z
4,01.15Z,Culture du tabac,1.15,1.1,A01Z,1,AZ1,AZ,AZ,AZ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01.15Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
727,96.09Z,Autres services personnels n.c.a.,96.09,96.0,S96Z,96,SZ3,SZ,RU,RU,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,96.09Z
728,97.00Z,Activités des ménages en tant qu'employeurs de...,97.00,97.0,T97Z,97,TZ0,TZ,RU,RU,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,97.00Z
729,98.10Z,Activités indifférenciées des ménages en tant ...,98.10,98.1,T98Z,98,TZ0,TZ,RU,RU,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.10Z
730,98.20Z,Activités indifférenciées des ménages en tant ...,98.20,98.2,T98Z,98,TZ0,TZ,RU,RU,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98.20Z


In [6]:
# cleaning naf2 for use later in my_SQL
col_to_drop = ['Unnamed: 10','Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 
               'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22']
naf2.drop(col_to_drop, axis = 1, inplace = True)

naf2


,SOUS- CLASSE,INTITULE DE LA NAF rév. 2,CLASSE,GROUPE,A 129,DIVISION,A 64,A 38,A 17,A 10
0,01.11Z,"Culture de céréales (à l'exception du riz), de...",1.11,1.1,A01Z,1,AZ1,AZ,AZ,AZ
1,01.12Z,Culture du riz,1.12,1.1,A01Z,1,AZ1,AZ,AZ,AZ
2,01.13Z,"Culture de légumes, de melons, de racines et d...",1.13,1.1,A01Z,1,AZ1,AZ,AZ,AZ
3,01.14Z,Culture de la canne à sucre,1.14,1.1,A01Z,1,AZ1,AZ,AZ,AZ
4,01.15Z,Culture du tabac,1.15,1.1,A01Z,1,AZ1,AZ,AZ,AZ
...,...,...,...,...,...,...,...,...,...,...
727,96.09Z,Autres services personnels n.c.a.,96.09,96.0,S96Z,96,SZ3,SZ,RU,RU
728,97.00Z,Activités des ménages en tant qu'employeurs de...,97.00,97.0,T97Z,97,TZ0,TZ,RU,RU
729,98.10Z,Activités indifférenciées des ménages en tant ...,98.10,98.1,T98Z,98,TZ0,TZ,RU,RU
730,98.20Z,Activités indifférenciées des ménages en tant ...,98.20,98.2,T98Z,98,TZ0,TZ,RU,RU


In [7]:
naf2_agreg

,Section,Libellé des sections,Code\nDivision,Intitulé
0,A,"AGRICULTURE, SYLVICULTURE ET PÊCHE",1,"Culture et production animale, chasse et servi..."
1,A,"AGRICULTURE, SYLVICULTURE ET PÊCHE",2,Sylviculture et exploitation forestière
2,A,"AGRICULTURE, SYLVICULTURE ET PÊCHE",3,Pêche et aquaculture
3,B,INDUSTRIES EXTRACTIVES,5,Extraction de houille et de lignite
4,B,INDUSTRIES EXTRACTIVES,6,Extraction d'hydrocarbures
...,...,...,...,...
83,S,AUTRES ACTIVITÉS DE SERVICES,95,Réparation d'ordinateurs et de biens personnel...
84,S,AUTRES ACTIVITÉS DE SERVICES,96,Autres services personnels
85,T,ACTIVITÉS DES MÉNAGES EN TANT QU'EMPLOYEURS ; ...,97,Activités des ménages en tant qu'employeurs de...
86,T,ACTIVITÉS DES MÉNAGES EN TANT QU'EMPLOYEURS ; ...,98,Activités indifférenciées des ménages en tant ...


In [8]:
naf2.columns = ['sous_classe', 'intitule_naf_rev_2', 'classe', 'groupe',
       'A_129', 'division', 'A_64', 'A_38', 'A_17', 'A_10']
naf2['sous_classe']=naf2['sous_classe'].str.replace(".","")
naf2

,sous_classe,intitule_naf_rev_2,classe,groupe,A_129,division,A_64,A_38,A_17,A_10
0,0111Z,"Culture de céréales (à l'exception du riz), de...",1.11,1.1,A01Z,1,AZ1,AZ,AZ,AZ
1,0112Z,Culture du riz,1.12,1.1,A01Z,1,AZ1,AZ,AZ,AZ
2,0113Z,"Culture de légumes, de melons, de racines et d...",1.13,1.1,A01Z,1,AZ1,AZ,AZ,AZ
3,0114Z,Culture de la canne à sucre,1.14,1.1,A01Z,1,AZ1,AZ,AZ,AZ
4,0115Z,Culture du tabac,1.15,1.1,A01Z,1,AZ1,AZ,AZ,AZ
...,...,...,...,...,...,...,...,...,...,...
727,9609Z,Autres services personnels n.c.a.,96.09,96.0,S96Z,96,SZ3,SZ,RU,RU
728,9700Z,Activités des ménages en tant qu'employeurs de...,97.00,97.0,T97Z,97,TZ0,TZ,RU,RU
729,9810Z,Activités indifférenciées des ménages en tant ...,98.10,98.1,T98Z,98,TZ0,TZ,RU,RU
730,9820Z,Activités indifférenciées des ménages en tant ...,98.20,98.2,T98Z,98,TZ0,TZ,RU,RU


In [9]:
naf2_agreg.columns = ['section', 'description_section', 'code_division', 'description_division']
naf2_agreg

,section,description_section,code_division,description_division
0,A,"AGRICULTURE, SYLVICULTURE ET PÊCHE",1,"Culture et production animale, chasse et servi..."
1,A,"AGRICULTURE, SYLVICULTURE ET PÊCHE",2,Sylviculture et exploitation forestière
2,A,"AGRICULTURE, SYLVICULTURE ET PÊCHE",3,Pêche et aquaculture
3,B,INDUSTRIES EXTRACTIVES,5,Extraction de houille et de lignite
4,B,INDUSTRIES EXTRACTIVES,6,Extraction d'hydrocarbures
...,...,...,...,...
83,S,AUTRES ACTIVITÉS DE SERVICES,95,Réparation d'ordinateurs et de biens personnel...
84,S,AUTRES ACTIVITÉS DE SERVICES,96,Autres services personnels
85,T,ACTIVITÉS DES MÉNAGES EN TANT QU'EMPLOYEURS ; ...,97,Activités des ménages en tant qu'employeurs de...
86,T,ACTIVITÉS DES MÉNAGES EN TANT QU'EMPLOYEURS ; ...,98,Activités indifférenciées des ménages en tant ...


In [10]:
#export to my SQL
# Name of database: 'match_project'

from sqlalchemy import create_engine 
import pymysql.cursors
import os
# import getpass
import urllib.parse


urllib.parse.quote_plus('!') # parse to 'remove-change' the weird characters in our password, here it is '!'
urllib.parse.quote_plus('é')
pw_raw = 'Alsimar10' # os.getenv('mysql_andy') # I had to put my.sql password
pw = urllib.parse.quote_plus(pw_raw)

#engine = sqlalchemy.create_engine(f"mysql://{username}:{password}@{host}:{port}")

connection_string = 'mysql+pymysql://root:' + pw + '@localhost:3306/' # we connect to our computer, if we connect to a company
                     # this is the driver                             # data base, then we need an IP address
engine = create_engine(connection_string)

with engine.connect() as conn:                                   # we are creating an alias
    conn.execute(f"CREATE DATABASE IF NOT EXISTS match_project") # we are creating a new name of database "match_project"

In [11]:
naf2.to_sql('naf2',engine, 'match_project', if_exists='replace', index=False) # 'pushing' the data
naf2_agreg.to_sql('naf2_agreg',engine, 'match_project', if_exists='replace', index=False) # 'pushing' the data

88

In [12]:
# exporting the two dataframes naf2 and naf2_agreg to csv 
naf2.to_csv("naf2.csv")
naf2_agreg.to_csv("naf2_agreg.csv")

In [22]:
df['bnf_names'].values

array(['Agriculture, sylviculture, pêche, élevage',
       'Espaces verts, jardineries et animaleries',
       'Industrie et distribution alimentaires',
       'Boissons, vins et spiritueux',
       'Industries extractives (hors énergie)',
       'Industries textiles, habillement, cuir',
       'Bijouterie, horlogerie, orfèvrerie',
       'Industries du bois (hors meubles)', 'Papier, carton, imprimerie',
       'Emballage', 'Industries chimiques et parachimiques',
       "Produits d'hygiène parfums et cosmétiques", 'Parapharmacie',
       'Industrie pharmaceutique. Médicament. Biotechnologies',
       'Métallurgie et travail des métaux',
       "Biens d'équipement industriels (hors électronique)",
       'Matériels de précision, optique, mesure',
       'Matériels de transport. Défense', 'Automobile',
       'Equipement de la maison, mobilier professionnel',
       'Jouets, cadeaux', "Artisanat et métiers d'art", 'Luxe',
       'Construction', 'Second oeuvre', 'Commerce, distribution',

In [35]:
len(df['bnf_names'].values)

46

In [32]:
list(naf2_agreg['description_division'].values)

['Culture et production animale, chasse et services annexes',
 'Sylviculture et exploitation forestière',
 'Pêche et aquaculture',
 'Extraction de houille et de lignite',
 "Extraction d'hydrocarbures",
 'Extraction de minerais métalliques',
 'Autres industries extractives',
 'Services de soutien aux industries extractives',
 'Industries alimentaires',
 'Fabrication de boissons',
 'Fabrication de produits à base de tabac',
 'Fabrication de textiles',
 "Industrie de l'habillement",
 'Industrie du cuir et de la chaussure',
 "Travail du bois et fabrication d'articles en bois et en liège, à l'exception des meubles ; fabrication d'articles en vannerie et sparterie",
 'Industrie du papier et du carton',
 "Imprimerie et reproduction d'enregistrements",
 'Cokéfaction et raffinage',
 'Industrie chimique',
 'Industrie pharmaceutique',
 'Fabrication de produits en caoutchouc et en plastique',
 "Fabrication d'autres produits minéraux non métalliques",
 'Métallurgie',
 "Fabrication de produits métal

In [27]:
!pip install fuzzywuzzy

  Obtaining dependency information for fuzzywuzzy from https://files.pythonhosted.org/packages/43/ff/74f23998ad2f93b945c0309f825be92e04e0348e062026998b5eefef4c33/fuzzywuzzy-0.18.0-py2.py3-none-any.whl.metadata


In [28]:
# To use to compare the columns
from fuzzywuzzy import process

C:\Users\Javier\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [33]:
# Sample data
col1 = list(df['bnf_names'].values)

col2 = list(naf2_agreg['description_division'].values)

# Initialize an empty list to store the connections
connections = []

# Loop through each item in col1
for item1 in col1:
    # Find the most similar item in col2
    match, score = process.extractOne(item1, col2)
    connections.append((item1, match, score))

# Create a DataFrame from the connections list
df_connections = pd.DataFrame(connections, columns=['col1', 'col2_match', 'similarity_score'])

# Display the DataFrame
print(df_connections)


                                                 col1  \
0           Agriculture, sylviculture, pêche, élevage   
1           Espaces verts, jardineries et animaleries   
2              Industrie et distribution alimentaires   
3                        Boissons, vins et spiritueux   
4               Industries extractives (hors énergie)   
5              Industries textiles, habillement, cuir   
6                  Bijouterie, horlogerie, orfèvrerie   
7                   Industries du bois (hors meubles)   
8                          Papier, carton, imprimerie   
9                                           Emballage   
10              Industries chimiques et parachimiques   
11          Produits d'hygiène parfums et cosmétiques   
12                                      Parapharmacie   
13  Industrie pharmaceutique. Médicament. Biotechn...   
14                  Métallurgie et travail des métaux   
15  Biens d'équipement industriels (hors électroni...   
16            Matériels de préc

In [34]:
df_connections

,col1,col2_match,similarity_score
0,"Agriculture, sylviculture, pêche, élevage",Pêche et aquaculture,86
1,"Espaces verts, jardineries et animaleries",Pêche et aquaculture,86
2,Industrie et distribution alimentaires,"Culture et production animale, chasse et servi...",86
3,"Boissons, vins et spiritueux","Culture et production animale, chasse et servi...",86
4,Industries extractives (hors énergie),Industries alimentaires,86
5,"Industries textiles, habillement, cuir",Industries alimentaires,86
6,"Bijouterie, horlogerie, orfèvrerie",Industrie chimique,45
7,Industries du bois (hors meubles),Travail du bois et fabrication d'articles en b...,86
8,"Papier, carton, imprimerie",Imprimerie et reproduction d'enregistrements,86
9,Emballage,Métallurgie,63


In [38]:
print(df_connections['col2_match'])


0                                  Pêche et aquaculture
1                                  Pêche et aquaculture
2     Culture et production animale, chasse et servi...
3     Culture et production animale, chasse et servi...
4                               Industries alimentaires
5                               Industries alimentaires
6                                    Industrie chimique
7     Travail du bois et fabrication d'articles en b...
8          Imprimerie et reproduction d'enregistrements
9                                           Métallurgie
10    Culture et production animale, chasse et servi...
11                                 Pêche et aquaculture
12                             Industrie pharmaceutique
13                             Industrie pharmaceutique
14                                          Métallurgie
15                           Extraction d'hydrocarbures
16                              Fabrication de boissons
17       Services de soutien aux industries extr

In [18]:
df_connections.columns = ['bnf_names', 'description_division', 'similarity_score']
df_connections

,bnf_names,description_division,similarity_score
0,"Agriculture, sylviculture, pêche, élevage",Pêche et aquaculture,86
1,"Espaces verts, jardineries et animaleries",Pêche et aquaculture,86
2,Industrie et distribution alimentaires,"Culture et production animale, chasse et servi...",86
3,"Boissons, vins et spiritueux","Culture et production animale, chasse et servi...",86
4,Industries extractives (hors énergie),Industries alimentaires,86
5,"Industries textiles, habillement, cuir",Industries alimentaires,86
6,"Bijouterie, horlogerie, orfèvrerie",Industrie chimique,45
7,Industries du bois (hors meubles),Travail du bois et fabrication d'articles en b...,86
8,"Papier, carton, imprimerie",Imprimerie et reproduction d'enregistrements,86
9,Emballage,Métallurgie,63


In [19]:
# pushing the agreg_to_bnf
df_connections.to_sql('agreg_to_bnf',engine, 'match_project', if_exists='replace', index=False) # 'pushing' the data

46

In [20]:
# exporting the dataframe df_connections to csv 
df_connections.to_csv("agreg_to_bnf.csv")

In [21]:
df_connections

,bnf_names,description_division,similarity_score
0,"Agriculture, sylviculture, pêche, élevage",Pêche et aquaculture,86
1,"Espaces verts, jardineries et animaleries",Pêche et aquaculture,86
2,Industrie et distribution alimentaires,"Culture et production animale, chasse et servi...",86
3,"Boissons, vins et spiritueux","Culture et production animale, chasse et servi...",86
4,Industries extractives (hors énergie),Industries alimentaires,86
5,"Industries textiles, habillement, cuir",Industries alimentaires,86
6,"Bijouterie, horlogerie, orfèvrerie",Industrie chimique,45
7,Industries du bois (hors meubles),Travail du bois et fabrication d'articles en b...,86
8,"Papier, carton, imprimerie",Imprimerie et reproduction d'enregistrements,86
9,Emballage,Métallurgie,63
